In [ ]:
import ast
import datetime
import folium
import geopandas as gpd
import json
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import re
import requests
import seaborn as sns
import time
from datetime import timedelta
from scipy.spatial import ConvexHull
from shapely.geometry import Point, LineString, mapping
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from tqdm import tqdm

# data preparation #

# SPEED #

In [ ]:
#uploading the data

slow_speed = pd.read_csv(path)

In [ ]:
#group speed data by MMSI and aggregate relevant columns
grouped_slow_speed = slow_speed.groupby("MMSI").agg(
    
    IMO =("IMO", "first"),
    counts_slow_speed=("MMSI", "size"),
    sum_telco=("ss_near_telco_cable", "sum"),
    sum_power=("ss_near_power_cable", "sum"),
    sum_gas=("ss_near_gas_pipe", "sum"),

).reset_index()

In [ ]:
#drop duplicates based on MMSI and IMO
grouped_slow_speed = grouped_slow_speed.drop_duplicates(subset=["MMSI", "IMO"])

NULL SPEED

In [ ]:
null_speed = pd.read_csv('null_speed_near_infra.csv')

In [ ]:
#group speed data by MMSI and aggregate relevant columns
grouped_null_speed = null_speed.groupby("MMSI").agg(
    IMO =("IMO", "first"),
    counts_null_speed=("MMSI", "size"),
    sum_telco=("ns_near_telco_cable", "sum"),
    sum_power=("ns_near_power_cable", "sum"),
    sum_gas=("ns_near_gas_pipe", "sum"),

).reset_index()

In [ ]:
#drop duplicates based on MMSI and IMO
grouped_null_speed = grouped_null_speed.drop_duplicates(subset=["MMSI", "IMO"])

In [ ]:
# merging dataframes on MMSI
merged_df = pd.merge(grouped_slow_speed, grouped_null_speed, on="MMSI", suffixes=("_slow", "_null"), how="outer")
merged_df.head()

In [ ]:
#drop duplicates
merged_df = merged_df.drop_duplicates(subset=["MMSI", "IMO_null","IMO_slow"])

In [ ]:
# filling empty values with 0
merged_df.fillna(0, inplace=True)


In [ ]:
# columns to convert to int
cols_to_convert = [
    "IMO_slow",
    "counts_slow_speed",
    "sum_telco_slow",
    "sum_power_slow",
    "sum_gas_slow",

    "counts_null_speed",
    "sum_telco_null",
    "sum_power_null",
    "sum_gas_null",
]
# converting specified columns to int
merged_df[cols_to_convert] = merged_df[cols_to_convert].astype(int)

In [ ]:
# renamin columns
merged_df.rename(columns={ 'IMO_slow': 'IMO'}, inplace=True)


In [ ]:
# rename dataframe
speed_anomalies_agg_df = merged_df

# CHANGE OF COURSE #


In [ ]:
#upload course changes anomalies data
cc_df = pd.read_csv('large_changes_course_anomalies.csv')

In [ ]:
#grouping the data by MMSI and aggregating the necessary columns
grouped_cc = cc_df.groupby("MMSI").agg(
    counts_cc=("MMSI", "size"),
    sum_telco_cc=("cc_near_telco_cable", "sum"),
    sum_power_cc=("cc_near_power_cable", "sum"),
    sum_gas_cc=("cc_near_gas_pipe", "sum"),
   
).reset_index()

In [ ]:
# uploading zigzagging anomalies
zz_df = pd.read_csv('sliding_window_results_with_infra.csv')

In [ ]:
#grouping the data by MMSI and aggregating the necessary columns
grouped_zz = zz_df.groupby("MMSI").agg(
    counts_zz=("MMSI", "size"),
    sum_telco_zz=("zz_near_telco_cable", "sum"),
    sum_power_zz=("zz_near_power_cable", "sum"),
    sum_gas_zz=("zz_near_gas_pipe", "sum"),

).reset_index()

In [ ]:
# merging the two dataframes on MMSI
course_anomalies_df = pd.merge(grouped_cc, grouped_zz, on="MMSI", how="outer")


In [ ]:
# filling empty values with 0
course_anomalies_df.fillna(0, inplace=True)

In [ ]:
# merge speed anomalies aggregated and course anomalies aggregated dataframes on MMSI
course_and_speed_anomalies_df = pd.merge(speed_anomalies_agg_df, course_anomalies_df, on="MMSI", how="outer")
course_and_speed_anomalies_df.fillna(0, inplace=True)


In [ ]:
# dropping dulicates based on MMSI and IMO
course_and_speed_anomalies_df = course_and_speed_anomalies_df.drop_duplicates(subset=["MMSI", "IMO"])

# GOING DARK #

In [ ]:
# uploading going dark data
dark_df = pd.read_csv('going_dark_with_infra.csv')

In [ ]:
# grouping the data by MMSI and aggregating the necessary columns
grouped_dark = dark_df.groupby("MMSI").agg(
    counts_dark=("MMSI", "size"),
    sum_telco_dark=("dark_near_telco_cable", "sum"),
    sum_power_dark=("dark_near_power_cable", "sum"),
    sum_gas_dark=("dark_near_gas_pipe", "sum"),

).reset_index()

# ALL TOGETHER #

In [ ]:
# merging aggregated anomalies dataframe with going dark anomalies dataframe on MMSI
all_anomalies_df = pd.merge(course_and_speed_anomalies_df, grouped_dark, on="MMSI", how="outer")
all_anomalies_df.fillna(0, inplace=True)


In [ ]:
# tranforming IMO column to int
all_anomalies_df["IMO"] = all_anomalies_df["IMO"].astype(int)

In [ ]:
#drop duplicates based on MMSI and IMO
all_anomalies_df = all_anomalies_df.drop_duplicates(subset=["MMSI", "IMO"])

# static anomalies #


# flag of convienience #

In [ ]:
# uploading the main dataframe 
df_flag = pd.read_csv('fin_data_with_area.csv')

In [ ]:
# creating a list of flag of convenience countries with Russia too
flag_of_convience = [
    "Antigua and Barbuda",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bermuda",
    "Bolivia",
    "Cameroon",
    "Cayman Islands",
    "Comoros",
    "Cook Islands",
    "Curacao",
    "Cyprus",
    "Equatorial Guinea",
    "Eswatini",
    "Faroe Islands",
    "French International Ship Registry (FIS)",
    "Gabon",
    "German International Ship Registry (GIS)",
    "Georgia",
    "Gibraltar",
    "Honduras",
    "Jamaica",
    "Lebanon",
    "Liberia",
    "Malta",
    "Madeira",
    "Marshall Islands",
    "Mauritius",
    "Moldova",
    "Mongolia",
    "Myanmar",
    "North Korea",
    "Palau",
    "Panama",
    "San Marino",
    'Sao Tome & Principe',
    "Sierra Leone",
    "St Vincent",
    "Sri Lanka",
    "Tanzania",
    "Togo",
    "Vanuatu"
    "Russia",
    'Saint Kitts & Nevis'
]

In [ ]:
# creating a dataframe for flags of convenience
df_flags_subset = df_flag[['MMSI', 'IMO', 'FLAGNAME']]

In [ ]:
# cretaing new variable FOC with 1 if the flag is a flag of convenience, otherwise 0
df_flags_subset['FOC'] = df_flag['FLAGNAME'].apply(lambda x: 1 if x in flag_of_convience else 0)

In [ ]:
# dropping duplicates and resetting the index
df_flags_subset = df_flags_subset.drop_duplicates(subset=["MMSI", "IMO"])
df_flags_subset.reset_index(drop=True, inplace=True)

# MMSI CHANGE #

In [ ]:
# uploading the main dataframe
change_of_mmsi = pd.read_csv('fin_data_with_area.csv')

In [ ]:

# Grouping the data by IMO and counting unique MMSI
imo_counts = change_of_mmsi.groupby("IMO")["MMSI"].nunique().reset_index()

# filtering the IMO counts to find those with more than one unique MMSI
imo_multiple = imo_counts[imo_counts["MMSI"] > 1]



In [ ]:
# resetting the index
imo_multiple.reset_index(drop=True, inplace=True)

In [ ]:
# rename the column for clarity
imo_multiple.rename(columns={"MMSI": "MMSI_CHANGES"}, inplace=True)

In [ ]:
# creating a dataframe with oinly IMO
to_merge = pd.DataFrame(change_of_mmsi, columns=['IMO'])

In [ ]:
#merging the IMO multiple changes dataframe with the to_merge dataframe that =contains IMO and filling NaN values with 0
merged_change_imo = pd.merge(to_merge, imo_multiple, on="IMO", how="outer")
merged_change_imo.fillna(0, inplace=True)

In [ ]:
# merging dataframes on IMO
merged_FOC_and_MMSI = pd.merge(df_flags_subset, merged_change_imo, on="IMO", how="outer")
merged_FOC_and_MMSI.fillna(0, inplace=True)

In [ ]:
# drop duplicates based on MMSI and IMO
merged_FOC_and_MMSI = merged_FOC_and_MMSI.drop_duplicates(subset=["MMSI", "IMO"])

# OLDER VESSEL #

In [ ]:
# uploading data of main dataframe
old =  pd.read_csv('fin_data_with_area.csv')

In [ ]:
# creating dataframe with only relevant columns
new_old = old[['MMSI', 'IMO', 'BUILT']]

In [ ]:
# dropping duplicates
new_old = new_old.drop_duplicates(subset=["MMSI", "IMO"])


In [ ]:
# assigning binary variable OLDER15Y based on the year of built
current_year = datetime.datetime.now().year  
new_old["OLDER15Y"] = np.where(current_year - new_old["BUILT"] > 15, 1, 0)

In [ ]:
#merging  the dataframes
static_merged = pd.merge(merged_FOC_and_MMSI, new_old, on="MMSI", how="outer")

In [ ]:
# drop irrelevantt columns
new_static_merged = static_merged.drop(columns=["IMO_x", "IMO_y"])


In [ ]:
#dropping duplicates
new_static_merged = new_static_merged.drop_duplicates(subset=["MMSI", "IMO"])

# FINAL #

In [ ]:
# merging the dataframes to create the final dataframe
final_to_cluster = pd.merge(all_anomalies_df, new_static_merged, on="MMSI", how="outer")

In [ ]:
# filling NaN values with 0
final_to_cluster.fillna(0, inplace=True)

In [ ]:
# dropping duplicates
final_to_cluster = final_to_cluster.drop_duplicates(subset=["MMSI", "IMO"])
